# Introduction 

Ce projet concerne le Travail de Bachelor sur l'analyse de données en temps réel sur l'arbitrage du light-contact boxing. Le but est de signaler en temps réel les divergences de points attribués par les juges.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib.dates as mdates
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN

## Data exploration
Chargement du fichier CSV LCBA_scores.csv

Voici une visualisation du match_id 21096

In [2]:
# Charger le CSV
data = pd.read_csv('LCBA_scores.csv')

# Afficher les colonnes spécifiées
cols_to_display = ['score_id', 'red_point', 'blue_point', 'date_create_app', 'judge_id', 'match_id']
data[cols_to_display]

,score_id,red_point,blue_point,date_create_app,judge_id,match_id
0,1,0,0,NaN,114816,21089
1,2,0,0,NaN,114822,21089
2,3,0,0,NaN,114824,21089
3,4,0,0,NaN,114826,21093
4,5,0,0,NaN,114813,21093
...,...,...,...,...,...,...
38778,38952,1,0,2023-04-15 14:18:52.2+00,115226,22514
38779,38953,1,0,2023-04-15 14:18:51.485+00,114927,22514
38780,38954,1,0,2023-04-15 14:18:56.33+00,114927,22514
38781,38955,1,0,2023-04-15 14:18:56.889+00,114926,22514


A vu d'oeil on voit qu'il existe quelques modifications/correction à apporter. Nous allonsp prémièrement se concentrer sur un seul match_id. Nous allons voir s'il existe des lignes contenant des valeurs négatives.

In [3]:
# Trouver les lignes avec des valeurs négatives pour red_point et blue_point
negative_scores = data[(data['red_point'] < 0) | (data['blue_point'] < 0)].sort_values('score_id')

# Afficher les colonnes spécifiées pour les données avec match_id 2109
cols_to_display2 = ['score_id', 'red_point', 'blue_point', 'judge_id']
negative_scores.loc[negative_scores['match_id'] == 21096, cols_to_display2]

,score_id,red_point,blue_point,judge_id
393,394,0,-1,114819
427,428,-1,0,114819
428,429,-1,0,114819
444,445,-1,0,114813


Et maintenant s'il existe valeurs supérieur à 1.

In [4]:
# Trouver les lignes avec des valeurs positives supérieur à 1 pour red_point et blue_point
negative_scores = data[(data['red_point'] > 1) | (data['blue_point'] > 1)].sort_values('score_id')

# Afficher les colonnes spécifiées pour les données avec match_id 2109
cols_to_display2 = ['score_id', 'red_point', 'blue_point', 'judge_id']
negative_scores.loc[negative_scores['match_id'] == 21096, cols_to_display]

,score_id,red_point,blue_point,date_create_app,judge_id,match_id


Nous allons maintenant voir la forme (type) de nos données :

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38783 entries, 0 to 38782
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   score_id                  38783 non-null  int64  
 1   red_penalty               38783 non-null  int64  
 2   red_point                 38783 non-null  int64  
 3   blue_penalty              38783 non-null  int64  
 4   blue_point                38783 non-null  int64  
 5   date_create               38783 non-null  object 
 6   date_change               38783 non-null  object 
 7   judge_id                  38783 non-null  int64  
 8   match_id                  38783 non-null  int64  
 9   date_create_app           36786 non-null  object 
 10  uuid                      36786 non-null  object 
 11  judge_club_id             38783 non-null  int64  
 12  match_id.1                38783 non-null  int64  
 13  PalmaresDate              38783 non-null  object 
 14  winner

## Data preparation

### Conversion date
On constate qu'il considère les format de date comme des objets. On les convertit pour pour faciliter les prochains calculs sur la série temporelle. Le datetime64[ns] est un type de données utilisé dans Pandas pour représenter les dates et heures avec une précision jusqu'à la nanoseconde. 

In [6]:
cols_to_convert = ['date_create', 'date_change','date_create_app','PalmaresDate','PalmaresRealEndTime','PalmaresRealStartTime','open_time']
data[cols_to_convert] = data[cols_to_convert].apply(pd.to_datetime)


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38783 entries, 0 to 38782
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   score_id                  38783 non-null  int64              
 1   red_penalty               38783 non-null  int64              
 2   red_point                 38783 non-null  int64              
 3   blue_penalty              38783 non-null  int64              
 4   blue_point                38783 non-null  int64              
 5   date_create               38783 non-null  datetime64[ns, UTC]
 6   date_change               38783 non-null  datetime64[ns, UTC]
 7   judge_id                  38783 non-null  int64              
 8   match_id                  38783 non-null  int64              
 9   date_create_app           36786 non-null  datetime64[ns, UTC]
 10  uuid                      36786 non-null  object             
 11  judge_club_id  

### Suppresssion des lignes négatives

Nous avons constaté précédement qu'il y avait des lignes négatives. Il ne faut pas les considérer. Une ligne négative est créée pour un éventuel réquilibrage si le juge attribue un point par erreur.

In [8]:
# Trouver les lignes avec des valeurs négatives pour red_point et blue_point
negative_scores = data[(data['red_point'] < 0) | (data['blue_point'] < 0)].sort_values('score_id')

# Afficher les colonnes spécifiées pour les données avec match_id 2109
cols_to_display2 = ['score_id', 'red_point', 'blue_point', 'judge_id']
negative_scores.loc[negative_scores['match_id'] == 21096, cols_to_display2]

,score_id,red_point,blue_point,judge_id
393,394,0,-1,114819
427,428,-1,0,114819
428,429,-1,0,114819
444,445,-1,0,114813


Nous allons donc cherche donc chercher 
- la ligne précèdente 
- la ligne suivante 
- la différence de temps avec la ligne précèdente
- la différence de temps avec la ligne suivante
- la ligne identifiante (score_id) qui est la plus proche

In [ ]:
# Filtre pour le match_id 21096
data = data[data['match_id'] == 21096]

# Parcourir les lignes avec des scores négatifs
for _, row in negative_scores.iterrows():
    score_id = row['score_id']
    judge_id = row['judge_id']
    match_id = row['match_id']

    for _, row_inner in negative_scores.iterrows():
        score_id_inner = row_inner['score_id']
        judge_id_inner = row_inner['judge_id']
        match_id_inner = row_inner['match_id']

        # Trouver les deux lignes les plus proches en termes de date_create_app avec le même judge_id et des scores positifs
        before_rows = data[(data['match_id'] == match_id) & (data['judge_id'] == judge_id) & (data['date_create_app'] < row_inner['date_create_app']) & ((data['red_point'] > 0) | (data['blue_point'] > 0))].sort_values('date_create_app', ascending=False)
        if not before_rows.empty:
            before_row = before_rows.iloc[0]
            time_diff_before = row_inner['date_create_app'] - before_row['date_create_app']
        else:
            before_row = None
            time_diff_before = None

        after_rows = data[(data['match_id'] == match_id) & (data['judge_id'] == judge_id) & (data['date_create_app'] > row_inner['date_create_app']) & ((data['red_point'] > 0) | (data['blue_point'] > 0))].sort_values('date_create_app')
        if not after_rows.empty:
            after_row = after_rows.iloc[0]
            time_diff_after = after_row['date_create_app'] - row_inner['date_create_app']
        else:
            after_row = None
            time_diff_after = None

        # Comparaison des valeurs de time_diff_before et time_diff_after
        if time_diff_before is not None and time_diff_after is not None:
            min_value = min(time_diff_before.total_seconds(), time_diff_after.total_seconds())
            if min_value == time_diff_before.total_seconds():
                min_row = before_row
            else:
                min_row = after_row
        elif time_diff_before is not None:
            min_row = before_row
        elif time_diff_after is not None:
            min_row = after_row
        else:
            min_row = None

        # Récupération du score_id de la ligne avec la valeur minimale
        if min_row is not None:
            min_score_id = min_row['score_id']
        else:
            min_score_id = None

        # Ajout des informations dans les colonnes correspondantes
        negative_scores.at[row_inner.name, 'before_row'] = before_row['score_id'] if before_row is not None else None
        negative_scores.at[row_inner.name, 'after_row'] = after_row['score_id'] if after_row is not None else None
        negative_scores.at[row_inner.name, 'time_diff_before'] = time_diff_before.total_seconds() if time_diff_before is not None else None
        negative_scores.at[row_inner.name, 'time_diff_after'] = time_diff_after.total_seconds() if time_diff_after is not None else None
        negative_scores.at[row_inner.name,'min_score_id'] = min_score_id


# Affichage des colonnes spécifiées pour les données avec match_id 21096
cols_to_display4 = ['score_id', 'red_point', 'blue_point', 'judge_id', 'before_row', 'after_row', 'time_diff_before', 'time_diff_after', 'min_score_id']
negative_scores.loc[negative_scores['match_id'] == 21096, cols_to_display4]

On constate un premier problème c'est que deux lignes nous retourne le meme min_score_id ! Les deux lignes se basent sur le même before_row et after_row !

Question : comment traiter le problème, quand même supprimer une autre ligne ? meme s'il est très eloigné ? Dans ce cas, il peut y avoir une baisse en qualité

Nous connaissons maintenant les lignes portant des valeurs négatives et sur quelle ligne elle a été compensée. Ce nettoyage permettra une meilleure précision lors de la détection des moments d'échanges.

In [10]:
match = data.loc[data['match_id'] == 21096, cols_to_display]
num_rows = len(match.index)
print("Nombre de lignes existantes :", num_rows)

Nombre de lignes existantes : 78


In [11]:
index_to_drop = data[(data['match_id'] == 21096) & ((data['red_point'] < 0) | (data['blue_point'] < 0))].index
data.drop(index=index_to_drop, inplace=True)

In [12]:
data = data.drop(data[(data['match_id'] == 21096) & ((data['red_point'] > 1) | (data['blue_point'] > 1))].index)
data.drop(index=index_to_drop, inplace=True)

KeyError: '[393, 427, 428, 444] not found in axis'

In [ ]:
match = data.loc[data['match_id'] == 21096, cols_to_display]
num_rows = len(match.index)
print("Nombre de lignes existantes :", num_rows)

In [ ]:
# Afficher les colonnes spécifiées pour les données avec match_id 21096
cols_to_display = ['score_id', 'red_point', 'blue_point', 'date_create_app', 'judge_id', 'match_id']
data.loc[data['match_id'] == 21096, cols_to_display]

## Modelling

In [ ]:

# Création d'une nouvelle colonne pour les points finaux de l'équipe rouge
data["red_final_points"] = data.apply(lambda x: (x["red_point"] - x["red_penalty"]) if x["red_point"] > 0 else (-1 * x["red_penalty"]), axis=1)

# Création d'une nouvelle colonne pour les points finaux de l'équipe bleue
data["blue_final_points"] = data.apply(lambda x: (x["blue_point"] - x["blue_penalty"]) if x["blue_point"] > 0 else (-1 * x["blue_penalty"]), axis=1)

# Calcul de la somme finale des points par match_id pour chaque équipe
red_points = data.groupby("match_id")["red_final_points"].sum().reset_index()
blue_points = data.groupby("match_id")["blue_final_points"].sum().reset_index()

# Renommage des colonnes pour plus de clarté
red_points = red_points.rename(columns={"red_final_points": "red_points"})
blue_points = blue_points.rename(columns={"blue_final_points": "blue_points"})

# Fusion des deux tableaux pour obtenir un tableau unique par match_id avec les points pour chaque équipe
match_points = pd.merge(red_points, blue_points, on="match_id")

# Affichage du tableau des points finaux par match_id pour chaque équipe
print(match_points)


In [ ]:

# Sélectionner les lignes correspondant au match_id 21096
match_rows = data.loc[data['match_id'] == 21096]

# Grouper les données par match_id et judge_id, puis calculer la somme des points rouges et bleus pour chaque groupe
scores_by_judge = match_rows.groupby(['match_id', 'judge_id']).sum()[['red_point', 'blue_point']]

# Afficher les scores par juge pour le match_id 21096
print(scores_by_judge)


In [ ]:
# Filtre pour le match_id 21096
data = data[data['match_id'] == 21096]

# Tri des données par date_create
data = data.sort_values('date_create_app')

# Regroupement des données par judge_id
grouped_data = data.groupby(['judge_id'])

# Boucle sur les groupes pour créer un graphe par judge_id
for name, group in grouped_data:
    # Création d'un graphe pour chaque judge_id
    fig, axs = plt.subplots(figsize=(15, 5))
    
    axs.plot(group['date_create_app'], group['blue_point'].cumsum(), label=f"Judge {name}")
    axs.plot(group['date_create_app'], group['red_point'].cumsum(), label=f"Judge {name} (red)", linestyle='--')

    # Configuration du graphe
    axs.set_xlabel("Date de création")
    axs.set_ylabel("Points")
    axs.legend()

    # Formatage de l'axe x pour afficher les minutes et secondes
    plt.xticks(rotation=45)
    axs.xaxis.set_major_formatter(mdates.DateFormatter('%M:%S'))

    # Ajustement des ticks de l'axe x pour afficher toutes les valeurs
    axs.set_xticks(group['date_plus_recente'])
    axs.set_xticklabels(group['date_plus_recente'].dt.strftime('%M:%S'))

    # Ajustement des ticks de l'axe y pour un écart de 1
    axs.set_yticks(range(0, max(group[['blue_point', 'red_point']].cumsum().max())+1))

    # Somme total des blue_points
    total_blue_points = group['blue_point'].sum()
    axs.text(0.5, 0.95, f"Total blue points sans les pénalités: {total_blue_points}", transform=axs.transAxes, ha="center")

    # Somme total des red_points
    total_red_points = group['red_point'].sum()
    axs.text(0.5, 0.9, f"Total red points sans les pénalités : {total_red_points}", transform=axs.transAxes, ha="center")

    plt.title(f"Points cumulés pour le juge {name}")
    plt.show()


In [ ]:
# Filtre pour le match_id 21096
data = data[data['match_id'] == 21096]

# Tri des données par date_create
data = data.sort_values('date_create_app')

# Regroupement des données par judge_id
grouped_data = data.groupby(['judge_id'])

# Création du graphique avec Plotly
fig = go.Figure()

# Boucle sur les groupes
for name, group in grouped_data:
    fig.add_trace(go.Scatter(x=group['date_create_app'], y=group['blue_point'].cumsum(),
                             mode='lines',
                             name=f"Judge {name} - Blue"))
    fig.add_trace(go.Scatter(x=group['date_create_app'], y=group['red_point'].cumsum(),
                             mode='lines',
                             name=f"Judge {name} - Red",
                             line=dict(dash='dash')))

# Configuration du graphique
fig.update_layout(title=f"Points cumulés - Match {'match_id'}",
                  xaxis_title="Date de création",
                  yaxis_title="Points",
                  xaxis_tickangle=-45,
                  xaxis_tickformat='%M:%S',
                  yaxis_tickmode='linear',
                  yaxis_tick0=0,
                  yaxis_dtick=1)

# Affichage des totaux des points bleus et rouges
total_blue_points = data['blue_point'].sum()
total_red_points = data['red_point'].sum()

fig.add_annotation(text=f"Total blue points sans les pénalités : {total_blue_points}",
                   xref='paper', yref='paper',
                   x=0.5, y=0.95, showarrow=False, font=dict(size=12))
fig.add_annotation(text=f"Total red points sans les pénalités : {total_red_points}",
                   xref='paper', yref='paper',
                   x=0.5, y=0.9, showarrow=False, font=dict(size=12))

fig.show()
